Taken from this excellent video series : https://www.youtube.com/watch?v=qaDe0qQZ5AQ&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=8

## load libraries

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dataset
import torchvision.transforms as transforms

import pandas as pd
import numpy as np


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
in_channels = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 1024
num_epochs = 5

In [ ]:
# datahead = pd.read_csv("../input/overheadmnist/version2/train.csv")

# datahead['label'].unique()

In [ ]:
# test defined cnn
class SimpleCNN(nn.Module):
    def __init__(self, in_channels=in_channels, num_classes=num_classes):
        super(SimpleCNN, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=in_channels, out_channels=8,kernel_size=(3,3),padding=(1,1), stride=(1,1))
        self.cnn2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), padding=(1,1), stride=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.fc1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self,x):
        out = F.relu(self.cnn1(x))
        out = self.pool(out)
        out = F.relu(self.cnn2(out))
        out = self.pool(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return out

## Importing external model

In [ ]:
# import sys

model = torchvision.models.vgg16(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
model


## Changing the model

In [ ]:
# this doesnt do anything
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    
    def forward(self, x):
        return x

In [ ]:
model.avgpool = Identity()
model.classifier = nn.Sequential(nn.Linear(512, 100), nn.ReLU(), nn.Linear(100,num_classes))
# model

# just like that, we modified default model!

In [ ]:
model.features[0] = nn.Conv2d(in_channels, 64, kernel_size=(3,3), padding=(1,1), stride=(1,1))

In [ ]:
model.to(device=device)
model

In [ ]:
# model = SimpleCNN().to(device=device)

In [ ]:
# testing
x = torch.randn(batch_size, in_channels, 32, 32).to(device=device)
y = model(x)
y.shape

The VGG model is too big to be used for something like mnist. so we will download cifar

## Defining a dataset

In [ ]:

class OverheadMnist(Dataset):
    def __init__(self, datapath):
        super(OverheadMnist, self).__init__()
        
        df = pd.read_csv(datapath)
        self.x = torch.from_numpy(df.iloc[:,1:].values)
        
        self.x = self.x.reshape(self.x.size(0), 1, 28, 28)
        self.x = self.x.float()
        self.y = torch.from_numpy(df.iloc[:,0].values)
        self.y = self.y.long()
        
        self.n_length = len(df)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_length

## Downloading the Cifar dataset

In [ ]:
# train_dataset = OverheadMnist('../input/overheadmnist/version2/train.csv')
# test_dataset = OverheadMnist('../input/overheadmnist/version2/test.csv')

train_dataset =dataset.CIFAR10(root='dataset/', train=True, transform=transforms.ToTensor(), download=True )
test_dataset = dataset.CIFAR10(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## training loop

In [ ]:
for epoch in range(num_epochs):
    
    for data, target in train_dataloader:
        data = data.to(device=device)
        target = target.to(device=device)
        
        pred = model(data)
        loss = loss_criterion(pred, target)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
    
    print(f"For epoch : {epoch} loss : {loss}")

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


In [ ]:
check_accuracy(train_dataloader, model)
check_accuracy( test_dataloader, model)